***
@author: jmeier<br>
date: 05. january 2024<br>
short description: 
-  Programm uses clincia addresses and community coordinates (from database)<br>
-  Uses googlemaps distance matrix API to get the distance and travel times from each community to all clinica<br>
-  Calculates travel times by car or transit at selectable depature_times<br>
-  Updates database with travel distances and times<br>
***

<div class="alert alert-box alert-danger"><b>Caveat:</b> Google distance matrix service is fee-based!<br> 
~50€ for calculation of travel times/distances for all bavarian communities with a population > 5000 (to all clinica with car and transit) <br>
<b>Google API key must be entered in file auth.py</b></div>

In [1]:
import googlemaps
import sqlite3
from datetime import datetime
import sys
import os
import importlib.util

# Absoluten Pfad zum gewünschten Ordner ermitteln
notebook_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "hidden"))

# Pfad zu sys.path hinzufügen
if notebook_dir not in sys.path:
    sys.path.append(notebook_dir)

# Modul dynamisch importieren
auth_path = os.path.join(notebook_dir, "auth.py")
spec = importlib.util.spec_from_file_location("auth", auth_path)
auth = importlib.util.module_from_spec(spec)
spec.loader.exec_module(auth)

from auth import google_auth # Personal Google API key is saved separately

In [15]:
conn = sqlite3.connect("../communities/communities.sqlite")
cur = conn.cursor()

In [16]:
# Loads personalGgoogle API key and initializes googlemaps Client with it
auth_google = auth()
gmaps = googlemaps.Client(key=auth_google)

In [17]:
# Sets the destinations all (clinica) and the depature time
destinations = ["LMU Klinikum Campus Großhadern Marchioninistraße 15, 81377 München, Germany",
                "Klinikum rechts der Isar der Technischen Universität München Ismaninger Str. 22, 81675 München, Germany",
                "Universitätsklinikum Würzburg Josef-Schneider-Straße 2, 97080 Würzburg, Germany",
                "Universitätsklinikum Erlangen Maximilianspl. 2, 91054 Erlangen, Germany",
                "Universitätsklinikum Regensburg Franz-Josef-Strauß-Allee 11, 93053 Regensburg, Germany",
                "Universitätsklinikum Augsburg Stenglinstraße 2, 86156 Augsburg, Germany"
                ]

departure_time = datetime(2024,1,11,7,0,0,0)

In [20]:
# Gets the communities  (with postleitzahl, name, latitude, longitude) where google_retrieved=0 (=no travel distance retrieved) and Land=9 (=Bayern) and size >5000
cur.execute("""SELECT Gemeinde.id, Gemeinde.postleitzahl, Gemeinde.name, Gemeinde.breitengrad, Gemeinde.längengrad, Gemeinde.bevölkerung_insgesamt, Land.id, Land.name
            FROM Gemeinde JOIN VB JOIN Kreis JOIN RB JOIN Land 
                ON  Gemeinde.vb_id  = VB.id
                AND VB.kreis_id     = Kreis.id
                AND Kreis.rb_id     = RB.id
                AND RB.land_id      = Land.id
            WHERE Gemeinde.google_retrieved = 0 AND Gemeinde.bevölkerung_insgesamt > 5000 AND Land.id = 9""")
communities = cur.fetchmany(10)

In [21]:
len(communities)

1

In [22]:
# If no valid travel is returned by Google (ZERO_RESULTS) the value "None" is returned, else the travel distance value
def distance(results, community, nr):
    if results["rows"][0]["elements"][nr]["status"] == "ZERO_RESULTS":
        print(community, " without results for #", nr)
        return None
    elif results["rows"][0]["elements"][nr]["status"] == "OK":
        return results["rows"][0]["elements"][nr]["distance"]["value"]
    else:
        print("Unknown status:")
        print(results["rows"][0]["elements"][nr]["status"])


In [23]:
# If no valid travel is returned by Google (ZERO_RESULTS) the value "None" is returned, else the travel time value
def time(results, community, nr):
    if results["rows"][0]["elements"][nr]["status"] == "ZERO_RESULTS":
        print(community, " without results for #", nr)
        return None
    elif results["rows"][0]["elements"][nr]["status"] == "OK":
        return results["rows"][0]["elements"][nr]["duration"]["value"]
    else:
        print("Unknown status:")
        print(results["rows"][0]["elements"][nr]["status"])

In [24]:
# Loops over all retrieved communities and sends request to google maps distance_matrix (1 community as origin and all clinica as destination)
# Gets results for travel by car and travel by transit.
# Parses the returned results (JSON Format) using functions "distance" and "time" (see above) and updates the database &sets google_retrieved = 1
for community in communities:
    #origins = community[1] + " " + community[2] + ", Germany"
    origins = str(community[3]) + "," + str(community[4])

    results_car = gmaps.distance_matrix(origins=origins, destinations=destinations, departure_time=departure_time, mode="driving")
    results_transit = gmaps.distance_matrix(origins=origins, destinations=destinations, departure_time=departure_time, mode="transit")
    
    google_thur7am_car_distance_lmu = distance(results_car, community, 0)
    google_thur7am_car_time_lmu = time(results_car, community, 0)
    google_thur7am_car_distance_tum = distance(results_car, community, 1)
    google_thur7am_car_time_tum = time(results_car, community, 1)
    google_thur7am_car_distance_würzburg = distance(results_car, community, 2)
    google_thur7am_car_time_würzburg = time(results_car, community, 2)
    google_thur7am_car_distance_erlangen = distance(results_car, community, 3)
    google_thur7am_car_time_erlangen = time(results_car, community, 3)
    google_thur7am_car_distance_regensburg = distance(results_car, community, 4)
    google_thur7am_car_time_regensburg = time(results_car, community, 4)
    google_thur7am_car_distance_augsburg = distance(results_car, community, 5)
    google_thur7am_car_time_augsburg = time(results_car, community, 5)

    google_thur7am_transit_distance_lmu = distance(results_transit, community, 0)
    google_thur7am_transit_time_lmu = time(results_transit, community, 0)
    google_thur7am_transit_distance_tum = distance(results_transit, community, 1)
    google_thur7am_transit_time_tum = time(results_transit, community, 1)
    google_thur7am_transit_distance_würzburg = distance(results_transit, community, 2)
    google_thur7am_transit_time_würzburg = time(results_transit, community, 2)
    google_thur7am_transit_distance_erlangen = distance(results_transit, community, 3)
    google_thur7am_transit_time_erlangen = time(results_transit, community, 3)
    google_thur7am_transit_distance_regensburg = distance(results_transit, community, 4)
    google_thur7am_transit_time_regensburg = time(results_transit, community, 4)
    google_thur7am_transit_distance_augsburg = distance(results_transit, community, 5)
    google_thur7am_transit_time_augsburg = time(results_transit, community, 5)

    cur.execute("""UPDATE Gemeinde SET 
                        google_thur7am_car_distance_lmu =?,             /*1*/
                        google_thur7am_car_time_lmu =?,
                        google_thur7am_car_distance_tum =?,
                        google_thur7am_car_time_tum =?,
                        google_thur7am_car_distance_würzburg =?,        /*5*/
                        google_thur7am_car_time_würzburg =?,
                        google_thur7am_car_distance_erlangen =?,
                        google_thur7am_car_time_erlangen =?,
                        google_thur7am_car_distance_regensburg =?,
                        google_thur7am_car_time_regensburg =?,          /*10*/
                        google_thur7am_car_distance_augsburg =?,
                        google_thur7am_car_time_augsburg =?,
                        google_thur7am_transit_distance_lmu =?,
                        google_thur7am_transit_time_lmu =?,
                        google_thur7am_transit_distance_tum =?,         /*15*/
                        google_thur7am_transit_time_tum =?,
                        google_thur7am_transit_distance_würzburg =?,
                        google_thur7am_transit_time_würzburg =?,
                        google_thur7am_transit_distance_erlangen =?,
                        google_thur7am_transit_time_erlangen =?,        /*20*/
                        google_thur7am_transit_distance_regensburg =?,
                        google_thur7am_transit_time_regensburg =?,
                        google_thur7am_transit_distance_augsburg =?,
                        google_thur7am_transit_time_augsburg =?,
                        google_retrieved =1                             /*25*/
                    WHERE id=?""",
                    (   google_thur7am_car_distance_lmu,                #1
                        google_thur7am_car_time_lmu,
                        google_thur7am_car_distance_tum,
                        google_thur7am_car_time_tum,
                        google_thur7am_car_distance_würzburg,           #5
                        google_thur7am_car_time_würzburg,
                        google_thur7am_car_distance_erlangen,
                        google_thur7am_car_time_erlangen,
                        google_thur7am_car_distance_regensburg,
                        google_thur7am_car_time_regensburg,             #10
                        google_thur7am_car_distance_augsburg,
                        google_thur7am_car_time_augsburg,
                        google_thur7am_transit_distance_lmu,
                        google_thur7am_transit_time_lmu,
                        google_thur7am_transit_distance_tum,            #15
                        google_thur7am_transit_time_tum,
                        google_thur7am_transit_distance_würzburg,
                        google_thur7am_transit_time_würzburg,
                        google_thur7am_transit_distance_erlangen,
                        google_thur7am_transit_time_erlangen,           #20
                        google_thur7am_transit_distance_regensburg,
                        google_thur7am_transit_time_regensburg,
                        google_thur7am_transit_distance_augsburg,
                        google_thur7am_transit_time_augsburg,
                                                                        #25
                    community[0]))         
    print(origins)
conn.commit()                         



(6848, '94078', 'Freyung, St', 48.808968, 13.54759, 7261, 9, 'Bayern')  without results for # 0
(6848, '94078', 'Freyung, St', 48.808968, 13.54759, 7261, 9, 'Bayern')  without results for # 0
(6848, '94078', 'Freyung, St', 48.808968, 13.54759, 7261, 9, 'Bayern')  without results for # 1
(6848, '94078', 'Freyung, St', 48.808968, 13.54759, 7261, 9, 'Bayern')  without results for # 1
(6848, '94078', 'Freyung, St', 48.808968, 13.54759, 7261, 9, 'Bayern')  without results for # 4
(6848, '94078', 'Freyung, St', 48.808968, 13.54759, 7261, 9, 'Bayern')  without results for # 4
48.808968,13.54759


In [12]:
results_transit

{'destination_addresses': ['Marchioninistraße 15, 81377 München, Germany',
  'Ismaninger Str. 22, 81675 München, Germany',
  'Josef-Schneider-Straße 2, 97080 Würzburg, Germany',
  'Maximilianspl. 2, 91054 Erlangen, Germany',
  'Franz-Josef-Strauß-Allee 11, 93053 Regensburg, Germany',
  'Stenglinstraße 2, 86156 Augsburg, Germany'],
 'origin_addresses': ['47.640846,10.126481'],
 'rows': [{'elements': [{'status': 'ZERO_RESULTS'},
    {'status': 'ZERO_RESULTS'},
    {'status': 'ZERO_RESULTS'},
    {'status': 'ZERO_RESULTS'},
    {'status': 'ZERO_RESULTS'},
    {'status': 'ZERO_RESULTS'}]}],
 'status': 'OK'}

In [13]:
cur.close()
conn.close()